In [ ]:
import sys
sys.path.insert(0, './bitcoin-taproot/test/functional/')
srcdir = "./bitcoin-taproot/src/" 

from test_framework.test_framework import BitcoinTestFramework
from test_framework.key import *
import binascii
import hashlib

# TODO: Add ExampleTest/setup_test/shutdown_test from utils.py

# N-of-N Musig

![test](images/musig_intro_0.jpg)


In this chapter, we introduce the interactive musig protocol which allows n-of-n participants to jointly create and spend taproot or tapscript output using aggregated Schnorr signatures. The advantage of Musig is that the public key and joint N-of-N spend are indistinguishable from a regular transaction on-chain. 

However, this is an interactive protocol which requires multiple rounds of communication between the wallets. We will proceed with an example execution of the Musig protocol below.

## Round 1 - Pubkey Generation

![test](images/musig_intro_1.jpg)

### 1) Key Generation & Pubkey Exchange

In [ ]:
keys = []
pubkeys = []

for i in range(3):
    private_key = ECKey()
    private_key.generate()
    public_key = private_key.get_pubkey()
    keys.append((private_key, public_key))
    # Pubkeys are shared amongst participating wallets.
    pubkeys.append(public_key)
    

### 2) Compute Key Challenges

In [ ]:
c_map, pk_musig = generate_musig_key(pubkeys)

# c_map is a map which contains challenge factors for each original public key.
for pk, c in c_map.items():
    print("Compressed Pubkey: ", pk.get_bytes().hex())
    print("Challenge: ", c.hex(), '\n')
    

### 3 - 4) Multiplication of Keys/Challenge and Aggregation of Public Keys

In [ ]:
keys_c = []
for idx, (private, public) in enumerate(keys):
    # All keys are now adjusted by the challenge factor
    private_c = private.mul(c_map[public])
    public_c = public.mul(c_map[public])
    keys_c.append((private_c, public_c))
    

## Round 2 - Signing & Nonce Aggregation

![test](images/musig_intro_2.jpg)

### 1) Individual (Secret) Nonce Generation

In [ ]:
nonce_map = {} # Pubkey: Nonce
nonce_points = []
for idx, (private_c, public_c) in enumerate(keys_c):
    nonce_map[public_c] = generate_schnorr_nonce()
    nonce_points.append(nonce_map[public_c].get_pubkey())
    print("Nonce %d: %s" %(idx, nonce_map[public_c].get_bytes().hex()))
    

### 2) Exchange & Aggreation of Nonce Points

In [ ]:
R_agg, negated = aggregate_schnorr_nonces(nonce_points)
print("R aggregated: ", R_agg.get_bytes().hex())
print("R_agg was negated: ", negated)


### 3) Negation of Individual Nonces

The negation boolean is important information for the signing round. If the aggregated nonce was negated, the individual nonces need to be negated as well during individual signing.

## Round 3 - Signing & Signature Aggregation

![test](images/musig_intro_3.jpg)

### 1) Individual Signing

In [ ]:
# TODO: Sign tx sighash and test mempool acceptance.
msg = hashlib.sha256(b'transaction').digest()
sigs = []
for private_c, public_c in keys_c:
    signature = private_c.sign_musig(nonce_map[public_c], negated, R_agg, pk_musig, msg)
    sigs.append(signature)


### 2) Signature Aggregation

In [ ]:
sig_agg = aggregate_musig_signatures(sigs)
print(pk_musig.verify_schnorr(sig_agg, msg))
